In [99]:
import torch
#clear memory better
with torch.no_grad():
    torch.cuda.empty_cache()
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
# from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
torch.cuda.is_available()

True

In [100]:
import se3_diffuse.utils as du

In [101]:
from data_rigid_diffuser import so3_diffuser
from data_rigid_diffuser import r3_diffuser
from data_rigid_diffuser import oneHot_diffuser
from scipy.spatial.transform import Rotation
from data_rigid_diffuser import rigid_utils as ru
import yaml
from data_rigid_diffuser.diffuser import FrameDiffNoise
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph import Helix4_Dataset, Make_KNN_MP_Graphs

In [102]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [103]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

# #find better way to incorporate coord_scale

# #needed
# N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
# C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')



In [104]:
# data_path_str  = 'data/h4_ca_coords.npz'
# test_limit = 1028
# rr = np.load(data_path_str)
# ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
# ca_coords.shape

# getting N-Ca, Ca-C vectors to add as typeI features
#apa = apart helices for val/train split
#tog = together helices for val/train split

#mode for tablet
apa_path_str  = 'data_npose/h4_apa_coords.npz'
tog_path_str  = 'data_npose/h4_tog_coords.npz'

#grab the first 3 atoms which are N,CA,C
test_limit = 256
rr = np.load(apa_path_str)
coords_apa = [rr[f] for f in rr.files][0][:test_limit,:]

rr = np.load(tog_path_str)
coords_tog = [rr[f] for f in rr.files][0][:test_limit,:]

In [105]:
B = 2
L=65
limit = 128
h4_trainData = Helix4_Dataset(coords_tog[:limit])
h4_valData = Helix4_Dataset(coords_apa[:limit])
train_dL = DataLoader(h4_trainData, batch_size=B, shuffle=True, drop_last=True)
val_dL   = DataLoader(h4_valData, batch_size=B, shuffle=True, drop_last=True)

In [106]:
L=65
limit = 128
prot_trainData = Data_Graph.ProteinBB_Dataset(coords_tog[:limit], n_nodes=128,
              n_atoms=5, coord_div=10, cast_type=torch.float32)
train_dL = DataLoader(prot_trainData, batch_size=B, shuffle=True, drop_last=True)

In [107]:
testiter = iter(train_dL)
bb_dict = next(testiter)

In [108]:
from data_rigid_diffuser import diffuser

In [109]:
config_path='data_rigid_diffuser/base.yaml'

        


In [110]:
fnd = FrameDiffNoise(config_path)


In [177]:
def circular_pe_encoding(n_nodes=128,embed_dim=12, cast_type=torch.float32):
    #positional encoding of node, all repeat every n_nodes
    i_array = np.arange(1,(embed_dim/2)+1)
    period = 2*np.pi
    wk = period/(n_nodes)*i_array**2
    t_array = np.arange(n_nodes)
    si = torch.tensor(np.sin(wk*t_array.reshape((-1,1))))
    ci = torch.tensor(np.cos(wk*t_array.reshape((-1,1))))
    pe = torch.stack((si,ci),axis=2).reshape(t_array.shape[0],embed_dim).type(cast_type)
    
    return pe

def monomer_null_knngraph(xyz,real_mask, k=2):
    """Takes in array of XYZ coordinates with null nodes repeated at termini.
    
    #Parameters
    real_mask: mask determining residues of real and null nodes
    
    Returns Graph that has a linear connections between all nodes indices and knn graph based on real nodes distance"""
    
    #make knn_graph from real nodes only
    #THERE ARE k PREDECESSORS FOR EACH NODE, SO CHECK DST 
    g_knn=dgl.knn_graph(xyz[real_mask],k=k,exclude_self=True) #this will make a batched 3d tensor if youwant

    #prepare adjacent node graph that connects real to null and (all nodes)
    n_nodes = xyz.shape[0]


    #create arrays to shift node indices_ from knn_graph based at zero back to original number
    knn_shift = torch.zeros((n_nodes,))

    #use mask to assign nodes as real or null
    #pulls the nodes out, as the real nodes will be assigned 0-len(real_nodes), and len(real_nodes) to end
    #like what happens with g_knn graph creation, where index of real_nodes_src convert to node numbering in graph
    #then null_nodes_src indexing + (len(real_nodes)) is equal to null node numbering in graph
    node_index =  torch.arange(n_nodes)
    real_nodes = node_index[real_mask==True]
    null_nodes = node_index[real_mask==False]

    # add null nodes to the end
    g_knn.add_nodes(len(null_nodes))
    #assign shifts as described above, from creating the graph will real nodes first then adding null nodes
    #the real nodes will be assigned 0-len(real_nodes), and len(real_nodes) to end
    knn_shift[:len(real_nodes)] = real_nodes
    knn_shift[len(real_nodes):] = null_nodes

    #convert the edges of knn_real+graph with adjacent graph added on top using knn_shift
    #to a new src, dst graph (src_conv,dst_conv) that refers to the original node numbering
    g_knn_src, g_knn_dst = g_knn.edges()

    conv_src = torch.ones_like(g_knn_src,dtype=torch.int)
    conv_dst = torch.ones_like(g_knn_dst,dtype=torch.int)

    #index i of src_knn_shift = (new knn_graph node numbering, value at )
    #(old graph node number) =  src_knn_shift[i] 
    #the real nodes will be assigned 0-len(real_nodes), and len(real_nodes) to end for null nodes
    #set the real nodes first, then the null nodes
    for i in range(real_nodes.shape[0]):
        conv_src[g_knn_src==i] = knn_shift[i]
        conv_dst[g_knn_dst==i] = knn_shift[i]

    for j in range(1,null_nodes.shape[0]+1):
        conv_src[g_knn_src==(i+j)] = knn_shift[i+j]
        conv_dst[g_knn_dst==(i+j)] = knn_shift[i+j]



    g = dgl.graph((conv_src,conv_dst))

    src_start = torch.arange(n_nodes,dtype=torch.int) #positive direction
    dst_start = torch.roll(src_start,-1) #negative dir
    #add adjacent graph on top, simple remove repeats
    g.add_edges(src_start,dst_start)
    g_out = dgl.to_simple(g)
    return g_out

def decirc(graph_in):
    """Remove circular connections. Last node to first node"""
    src,dst = graph_in.edges()

    circfw = (src==graph_in.nodes().max()) & (dst==0) 
    circbw = (src==0) & (dst==graph_in.nodes().max())
    both_circ = circfw | circbw
    
    src_out = src[~both_circ]
    dst_out = dst[~both_circ]
    
    return dgl.graph((src_out,dst_out))

def concat_monomer_graphs(graph_list):
    """Add single connection between each graph"""
    
    batched_graph = dgl.batch([decirc(g) for g in graph_list])
    src = torch.zeros((len(graph_list),),dtype=torch.int64) 
    dst = torch.zeros_like(src)
    
    #recirc concats
    nindex = 0
    src[nindex] = 0
    dst[nindex] = len(batched_graph.nodes())-1 #
    
    
    for i in range(1,len(graph_list)):
        src[i] = graph_list[nindex].nodes().max()+nindex
        dst[i] = src[i]+1
        nindex = src[i]
    
    
    batched_graph.add_edges(src,dst)
    batched_graph.add_edges(dst,src)
    
    return batched_graph


In [411]:
#this should probably be already transferred to GPU for speed
class Make_nullKNN_MP_Graphs():
    
    #8 long positional encoding
    NODE_FEATURE_DIM_0 = 12
    EDGE_FEATURE_DIM = 1 # 0 or 1 primary seq connection or not
    NODE_FEATURE_DIM_1 = 2
    
    def __init__(self, KNN=30, mp_stride=4, n_nodes=128, coord_div=10, 
                       cast_type=torch.float32, channels_start=32,
                       ndf1=6, ndf0=32,cuda=True):
        
        self.KNN = KNN
        self.n_nodes = n_nodes
        self.pe = circular_pe_encoding(n_nodes=n_nodes,embed_dim=12, cast_type=torch.float32)
        self.mp_stride = mp_stride
        self.null_stride = mp_stride*2
        self.cast_type = cast_type
        self.channels_start = channels_start
        
        self.cuda = cuda
        self.ndf1 = ndf1 #awkard adding of nodes features to mpGraph
        self.ndf0 = ndf0
        
        
    def create_and_batch(self, bb_dict, print_out=False):

        graphList = []
        mpGraphList = []
        mpRevGraphList = []
        mpSelfGraphList = []
        
        for j, caXYZ in enumerate(bb_dict['bb_noised']['CA']):
            #round nodes to be real (1) or (null 0)
            #get null node indices from mask
            real_nodes_feats = torch.round(bb_dict['real_nodes_noise'][j]).clamp(0,1)
            real_nodes_mask = real_nodes_feats.sum(-1)>1.99
            
            #make a knn graph form the real nodes only
            graph = monomer_null_knngraph(caXYZ, real_nodes_mask, k=self.KNN)
            graph.ndata['pe'] = torch.cat((self.pe,real_nodes_feats),dim=-1)
            graph.ndata['pos'] = caXYZ
            graph.ndata['bb_ori'] = torch.cat((bb_dict['bb_noised']['N_CA'][j], bb_dict['bb_noised']['C_CA'][j]),axis=1)
            graph.ndata['real_nodes_mask']=real_nodes_mask
            
            #gather edge data from all possible noised edges produced
            gsrc, gdst = graph.edges()
            num_edges = len(gsrc)

            #adjacent AA are one apart, or the loop connection from zero node to the last node
            adj_nodes_mask = ((torch.abs(gsrc-gdst)==1) | (torch.abs(gsrc-gdst)==len(gsrc)-1)) 

            #actually we need to determine
            null_nodes = torch.arange(self.n_nodes,dtype=torch.int)[~real_nodes_mask]
            real_nodes = torch.arange(self.n_nodes,dtype=torch.int)[real_nodes_mask]

            #broadcast each src/dst node against all null nodes, if any match along node dimension is a null edge
            gsrc_compare = (gsrc[:,None] - null_nodes[None,:])
            gdst_compare = (gdst[:,None] - null_nodes[None,:])
            null_edges_src_ind = torch.where(gsrc_compare==0,True,False).any(dim=1)
            null_edges_dst_ind = torch.where(gdst_compare==0,True,False).any(dim=1)

            #broadcast each src/dst node against all real or null nodes, if any match along node dimension is a null edge
            gsrc_compare = (gsrc[:,None] - real_nodes[None,:])
            gdst_compare = (gdst[:,None] - real_nodes[None,:])
            real_edges_src_ind = torch.where(gsrc_compare==0,True,False).any(dim=1)
            real_edges_dst_ind = torch.where(gdst_compare==0,True,False).any(dim=1)
            #null edges connect to any null nodes (|), real edges must both be real (&)
            null_edges = null_edges_src_ind | null_edges_dst_ind
            real_edges = real_edges_src_ind & real_edges_dst_ind
            adj_real = adj_nodes_mask&real_edges

            #is this needed, taken care of a node

            EDGE_DIM=1

            ###
#             enoised = noised_dict['edge_cons'][j]
#             enoised = enoised.reshape((-1,EDGE_DIM,2))[:num_edges]
            edata = torch.zeros(gsrc.shape+(EDGE_DIM,))
            # oh1_mask = torch.tensor([1,0,0]).unsqueeze(-1)
            # oh2_mask = torch.tensor([0,1,0]).unsqueeze(-1)
            # oh3_mask = torch.tensor([0,0,1]).unsqueeze(-1)


            # #needs to add - /+ N/C directon for this data????, or is this covered by pe encoding
            # edata[real_edges] = torch.gather(enoised[real_edges],2,oh2_mask.repeat(real_edges.sum(),1,1))[:,:,0]
            # edata[adj_real] = torch.gather(enoised[adj_real],2,oh1_mask.repeat(adj_real.sum(),1,1))[:,:,0]
            # edata[null_edges] = torch.gather(enoised[null_edges],2,oh3_mask.repeat(null_edges.sum(),1,1))[:,:,0]

            # edge_dir = torch.ones_like(gsrc,dtype=torch.long) #NotC direction off AA sequence
            # edge_dir[gdst<gsrc]=-1 # #NotC; reverse direction 
            # edata[adj_nodes_mask]=edata[adj_nodes_mask]*edge_dir

            graph.edata['con'] = edata

            #way to get real_edges and null edges in loop code
            # null_edges = torch.zeros_like(gsrc)
            # for i,x in enumerate(gsrc):
            #     if gsrc[i] in null_nodes or gdst[i] in null_nodes:
            #         null_edges[i] = 1

            # real_edges = torch.zeros_like(gsrc)
            # for i,x in enumerate(gsrc):
            #     if gsrc[i] in real_nodes and gdst[i] in real_nodes:
            #         real_edges[i] = 1

            mp_list = torch.zeros((len(list(range(0,len(real_nodes), self.mp_stride)))),caXYZ.shape[1])

            new_src = torch.tensor([],dtype=torch.int)
            new_dst = torch.tensor([],dtype=torch.int)


            new_src_rev = torch.tensor([], dtype=torch.int)
            new_dst_rev = torch.tensor([], dtype=torch.int)

            #create
            i=0#mp list counter
            for real_index in range(0,len(real_nodes), self.mp_stride):
                x = real_nodes[real_index] #convert to match torch.int from
                src, dst = graph.in_edges(x) #dst repeats x, this grab null nodes too
                
                n_tot = torch.cat((x.unsqueeze(0),src)) #add x to node list
                mp_list[i] = caXYZ[n_tot].sum(axis=0)/n_tot.shape[0]
                mp_node = i + graph.num_nodes() #add midpoints nodes at end of graph
                #define edges between midpoint nodes and nodes defining midpoint for midpointGraph

                new_src = torch.cat((new_src,n_tot))
                new_dst = torch.cat((new_dst,
                                     (torch.tensor(mp_node,dtype=torch.int).unsqueeze(0).repeat(n_tot.shape[0]))))
                
#                 #and reverse graph for coming off
#                 new_src_rev = torch.cat((new_src_rev,
#                                          (torch.tensor(mp_node,dtype=torch.int).unsqueeze(0).repeat(n_tot.shape[0]))))
#                 new_dst_rev = torch.cat((new_dst_rev,n_tot))

                i+=1
                
            #remove extra null nodes from .in_edges call
            
            #remove edges that are null node connections,
            #dst are the midpoint nodes for mpGraph, src are mp nodes for mpGraphRev
            #only remove non-mp nodes
            
            real_mask_rem1 = torch.isin(new_src,real_nodes)
#             real_mask_rem2 = torch.isin(new_dst_rev,real_nodes) 
            new_src = new_src[real_mask_rem1]
            new_dst = new_dst[real_mask_rem1]
#             new_src_rev = new_src_rev[real_mask_rem2]
#             new_dst_rev = new_dst_rev[real_mask_rem2]
                
                
            #collapse collected null nodes onto null mp of contingous sections 
            end_p = ((null_nodes.roll(1)-null_nodes)==-1) #consecutive are equal to negative one (look right)
            start_p = ((null_nodes.roll(-1)-null_nodes)==1) #consecutive are equal to one (look left)
            startend = (start_p != end_p) #remove overlap of interior consecutive nodes
            start = start_p == startend #just get the starts
            end  = end_p == startend #just get the ends
            si = torch.arange(len(start),dtype=torch.int)[start]# indices of start of consecutive nodes
            ei = torch.arange((len(end)),dtype=torch.int)[end] # indices of end of cone

            #connect first and last groups if approriate
            if null_nodes[0]==0 and null_nodes[-1]==self.n_nodes-1:
                #roll last group across barrier
                roll_con = len(start)-si[-1]
                null_nodes = null_nodes.roll(int(roll_con))
                #update end index and start index by roll and remove groups (one from end)
                #add zero to start and remove last start (rolled)
                ei = (ei+roll_con)[:-1]
                sic=torch.zeros_like(si[1:])
                sic[1:] = si[1:-1]+roll_con
                si = sic

            mp_list_null  = torch.ones((si.shape[0],caXYZ.shape[1]))*-1e9

            counter_mp_index = 0 #mp list counter
            counter_mp_node = 0 #counter for number of mp nodes added, 
            tot_indices = si.shape[0]
            while counter_mp_index < tot_indices:
                #coll
                #print(null_nodes[si[counter_mp_index]:ei[counter_mp_index]+1])
                n_tot = null_nodes[si[counter_mp_index]:ei[counter_mp_index]+1]
            #                 print(len(n_tot))
                while len(n_tot) <  mkg.mp_stride and counter_mp_index+1<tot_indices:
                    counter_mp_index=counter_mp_index+1
                    n_tot = torch.cat([n_tot,null_nodes[si[counter_mp_index]:ei[counter_mp_index]+1]],axis=0)

                mp_list_null[counter_mp_index] = caXYZ[n_tot].sum(axis=0)/n_tot.shape[0]
                mp_node = counter_mp_node + i + graph.num_nodes() #add midpoints nodes at end of graph
                counter_mp_node += 1

                #from null nodes to new mp_node
                new_src = torch.cat((new_src,n_tot))
                new_dst = torch.cat((new_dst,
                                     (torch.tensor(mp_node,dtype=torch.int).unsqueeze(0).repeat(n_tot.shape[0]))))
                #and reverse graph for coming off
                new_src_rev = torch.cat((new_src_rev,
                                         (torch.tensor(mp_node,dtype=torch.int).unsqueeze(0).repeat(n_tot.shape[0]))))
                new_dst_rev = torch.cat((new_dst_rev,n_tot))

                counter_mp_index=counter_mp_index+1

            mp_list_null = mp_list_null[(mp_list_null>-1e8).any(axis=1)]
            mp_list_real_null = torch.cat([mp_list,mp_list_null])
            mp_node_indx = torch.arange(0,i+len(mp_list_null)).type(torch.int)

            mpGraph = dgl.graph((new_src,new_dst))
            mp_pos = torch.cat((caXYZ,mp_list_real_null),axis=0).type(self.cast_type)

            mpGraph = dgl.graph((new_src,new_dst))
            mpGraph.ndata['pos'] = torch.cat((caXYZ,mp_list_real_null),axis=0).type(self.cast_type)
            
            #mp real/ null nodes
            mp_node_real_mask = torch.zeros(mp_list_real_null.shape[0],dtype=torch.int)
            mp_node_real_mask[:len(mp_list)] = 1
            mpGraph.ndata['mp_node_real_mask'] = torch.cat([real_nodes_mask,mp_node_real_mask])
            #match output shape of first transformer
            pe_mp = torch.cat((self.pe,torch.zeros((self.pe.shape[0], self.channels_start-self.pe.shape[1]))),axis=1)
            mpGraph.ndata['pe'] = torch.cat((pe_mp,pe_mp[mp_node_indx]))
            mpGraph.edata['con'] = torch.zeros((mpGraph.num_edges(),1))

            #mpGraph_rev = dgl.graph((new_src_rev,new_dst_rev))
            mpGraph_rev = dgl.graph((new_dst,new_src))
            mpGraph_rev.ndata['pos'] = torch.cat((caXYZ, mp_list_real_null),axis=0).type(self.cast_type)
            mpGraph_rev.ndata['pe'] = torch.cat((pe_mp,pe_mp[mp_node_indx]))
            mpGraph_rev.edata['con'] = torch.zeros((mpGraph_rev.num_edges(),1))
            mpGraph_rev.ndata['mp_node_real_mask'] = torch.cat([real_nodes_mask,mp_node_real_mask])

            #make graph for self interaction of midpoints
            v1,v2,edge_data, ind = define_graph_edges(len(mp_list_real_null))
            mpSelfGraph = dgl.graph((v1,v2))
            mpSelfGraph.edata['con'] = edge_data.reshape((-1,1))
            mpSelfGraph.ndata['pe'] = self.pe[mp_node_indx] #not really needed
            mpSelfGraph.ndata['pos'] = mp_list_real_null.type(self.cast_type)



            mpSelfGraphList.append(mpSelfGraph)
            mpGraphList.append(mpGraph)
            mpRevGraphList.append(mpGraph_rev)
            graphList.append(graph)
       
        return dgl.batch(graphList), dgl.batch(mpGraphList), dgl.batch(mpSelfGraphList), dgl.batch(mpRevGraphList)
        
        

In [412]:
noised_dict = fnd.forward(bb_dict,t_vec=np.ones((B,))*0.0001)
#edges are Batch, Length, KNN, 3, bool  
#each edge represents real adj, fake dist
mkg = Make_nullKNN_MP_Graphs()
 #real and direct connect, real and non-direct, false and direct # no false and not direct

In [413]:
mpSelfGraph,mpGraph,mpGraph_rev,graph = mkg.create_and_batch(noised_dict,print_out=True)
# np.arange(len(graph.nodes()))[graph.ndata['real_nodes_mask']]
# s,d = graph.edges()